[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1yHECdcu9zydeXmVU5r7smMPWIOE6kXAE)

# Echolocation Segment Extraction 





In [ ]:
import pandas as pd

In [ ]:
# datetime packages
from datetime import timezone
from datetime import datetime, timedelta

In [ ]:
# orca_hls_utils is a small package to facilitate the file extraction
# !pip install orca_hls_utils

In [ ]:
# this is a version which has an extra option to overwrite existing files which is handy while testing
!pip install git+git://github.com/orcasound/orca-hls-utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning git://github.com/orcasound/orca-hls-utils to /tmp/pip-req-build-dvq4q_yv
  Running command git clone -q git://github.com/orcasound/orca-hls-utils /tmp/pip-req-build-dvq4q_yv
  fatal: unable to connect to github.com:
  github.com[0: 140.82.114.4]: errno=Connection timed out

ERROR: Command errored out with exit status 128: git clone -q git://github.com/orcasound/orca-hls-utils /tmp/pip-req-build-dvq4q_yv Check the logs for full command output.


In [ ]:
import orca_hls_utils as ohu

ModuleNotFoundError: ignored

In [ ]:
from orca_hls_utils.DateRangeHLSStream import DateRangeHLSStream

### Reading Human Annotations

We will extract time stamps for echolocation from human annotations within the Orcasound website. The human annotations are stored in a Postgres Database on Heroku. Below we access an older dump of this database stored in a publicly accessible google sheet.

In [ ]:
# getting the file from google sheets
!wget  "https://docs.google.com/spreadsheets/d/1G3TxQVMUN3GIzhW7FEqLipQEK95E0GfwZE-qbnMHTy4/export?format=csv" -O human-detections.csv

In [ ]:
df = pd.read_csv('human-detections.csv', parse_dates=['timestamp'])

In [ ]:
df

In [ ]:
# select keys relevant to echolocation
key_list = ['echolocation', 'clicks', 'clicking', 'click']

In [ ]:
df_clean = df.dropna(axis=0, subset=['description'])

In [ ]:
df_key = df_clean.loc[df_clean['description'].str.contains('|'.join(key_list), case=False)]

In [ ]:
print('There are ' + str(df_key.shape[0]) + ' echolocation observations.')

In [ ]:
df_key.timestamp

In [ ]:
# dictionary for different hydrophone nodes
feed_dict = {1: 'orcasound_lab', 2: 'bush_point', 35: 'port_townsend'}

In [ ]:
# setting the stream base for the first echolocaiton example
stream_base = 'https://s3-us-west-2.amazonaws.com/streaming-orcasound-net/rpi_' + feed_dict[df_key.iloc[0]['feed_id']]
stream_base

### Downloading Segments for all Observations

We will use the `DateRangeHLSStream` function of the `orca_hls_utils` package.

In [ ]:
# convert to unix time
annot_unix_time = pd.to_datetime(df_key.timestamp.iloc[0]).replace(tzinfo=timezone.utc).timestamp()
annot_unix_time

In [ ]:
# using DateRangeHLSStream to extract a segment 60sec before the annot_unix_time

stream = DateRangeHLSStream(stream_base = stream_base,
    polling_interval = 60,
    start_unix_time = annot_unix_time - 60,
    end_unix_time = annot_unix_time,
    wav_dir = './',
    overwrite_output=True)

In [ ]:
stream.get_next_clip()

Setting some parameters for the segments to extract segments. 

In [ ]:
# number of seconds to go backwards and start from
lag = 60

# polling interval
polling_interval = 60

# output_path
OUTPUT_PATH = './'

In [ ]:
def download_segments(df_key, lag=60, polling_interval=60, output_path='./'):
  """ 
     download_segments is a function to downloads the segments for all observations in a data frame.
     The segments are downloaded in .wav format

     Inputs
     ------
     df_key: a pandas dataframe with observations of interest
     lag
     polling_interval
     OUTPUT_PATH: the path to store all segment files
     
  """
  for i in range(len(df_key)):
    annot_unix_time = pd.to_datetime(df_clicks.timestamp.iloc[i]).replace(tzinfo=timezone.utc).timestamp()
    stream_base = 'https://s3-us-west-2.amazonaws.com/streaming-orcasound-net/rpi_' + feed_dict[df_clicks.iloc[i]['feed_id']]
    stream = DateRangeHLSStream(stream_base = stream_base,
      polling_interval = polling_interval,
      start_unix_time = annot_unix_time - lag,
      end_unix_time = annot_unix_time,
      wav_dir = output_path,
      overwrite_output=True)
    stream.get_next_clip()


In [ ]:
# downloading all segments: this will take a while
download_segments(df_key)

Found 3514 folders in all for hydrophone
Found 1 folders in date range


live1664.ts: 188kB [00:00, 363kB/s]                            


live1665.ts: 188kB [00:00, 344kB/s]                            


live1666.ts: 188kB [00:00, 366kB/s]                            


live1667.ts: 188kB [00:00, 362kB/s]                            


...

live1860.ts: 106kB [00:00, 234kB/s]                            


live1861.ts: 106kB [00:00, 230kB/s]                            


live1862.ts: 106kB [00:00, 226kB/s]                            


live1863.ts: 106kB [00:00, 227kB/s]                             


KeyboardInterrupt: ignored

Preparing `.wav`'s for export.

In [ ]:
mkdir wavs

In [ ]:
mv *.wav wavs/

In [ ]:
ls

human-detections.csv  rpi-port-townsend_2020_07_03_17_35_18.wav  sample_data/


In [ ]:
!zip -r wavs.zip wavs

  adding: wavs/ (stored 0%)
  adding: wavs/rpi-port-townsend_2020_07_04_15_56_55.wav (deflated 12%)
  adding: wavs/rpi-orcasound-lab_2020_07_05_23_02_37.wav (deflated 51%)
  adding: wavs/rpi-bush-point_2020_06_18_15_17_41.wav (deflated 48%)
  adding: wavs/rpi-bush-point_2020_07_11_14_35_04.wav (deflated 56%)
  adding: wavs/rpi-bush-point_2020_09_01_17_23_25.wav (deflated 47%)
  adding: wavs/rpi-orcasound-lab_2020_10_20_05_01_33.wav (deflated 15%)
  adding: wavs/rpi-bush-point_2020_11_22_18_19_02.wav (deflated 18%)
  adding: wavs/rpi-port-townsend_2020_07_07_12_30_30.wav (deflated 22%)
  adding: wavs/rpi-bush-point_2020_07_11_17_01_13.wav (deflated 15%)
  adding: wavs/rpi-port-townsend_2020_07_03_17_35_18.wav (deflated 15%)
  adding: wavs/rpi-bush-point_2020_09_28_06_00_29.wav (deflated 14%)
  adding: wavs/rpi-orcasound-lab_2020_10_20_05_10_33.wav (deflated 15%)
  adding: wavs/rpi-orcasound-lab_2020_07_05_22_53_26.wav (deflated 45%)
  adding: wavs/rpi-port-townsend_2020_06_02_08_34_02.w